In [4]:
import pandas as pd
import re
import fitz
import os
import requests
from time import sleep

### <B> CODIGO PARA OBTENER LOS STUDENT CODES

In [5]:
with open("./codigos_uao.txt", "r") as f:
    codigos = f.read().splitlines()
print(len(codigos))

24754


#### <b> <font color="#ffffff"> CODIGO PARA DESCARGAR LOS PDFS

In [6]:
# def get_estados_de_cuenta(codes: list):
#     for i in codes:
#         try:
#             url = f"http://wlapp.uao.edu.co/reports/rwservlet?connrepuao_ice&jobtype=jt_uao_ice&report=Estado_Cta_Web_Uao10g.rdf&destype=cache&desformat=pdf&mi_cliente={i}"
#             response = requests.get(url)
#             with open(f"../data/uao_ctas/{i}_ctadt.pdf", "wb") as f:
#                 f.write(response.content)
#         except: pass

#### <font color = #ffffff> <b> CODIGO DE LA FUNCION PARA CREAR EL DATAFRAME

In [7]:
def create_data(archive_list: list):
    data = []
    del_files = 0
    for archivo in archive_list:
        try:
            doc = fitz.open(f"../data/uao_ctas/{archivo}")
            page1 = doc[0]

            # una lista de tuplas con las cordenadas de cada palabra y la palabra en si.
            words = page1.get_text("words")

            # editar el pdf para añadir el contenedor de los saldos.
            x0, y0, x1, y1 = 472,46,502,57
            rect = fitz.Rect(x0-10,y0-20,x1+62,y1)
            page1.draw_rect(rect, color=[0,1,1,0], overlay=False,width=0.5)

            # editar el pdf para añadir el contenedor del nombre y el programa.

            x02, y02, x12, y12 = 180,95,240,109
            rect2 = fitz.Rect(x02-150,y02-7,x12+350,y12+27)
            page1.draw_rect(rect2, color=[0,1,1,0],width=0.5)

        # TRABAJAMOS CON LOS DATOS QUE TIENEN RECTANGULOS
            saldo_words_list = [word for word in words if fitz.Rect(word[:4]) in rect]
            user_words_list = [word for word in words if fitz.Rect(word[:4]) in rect2]

            saldo_words = [word[4] for word in saldo_words_list]
            user_words = [word[4] for word in user_words_list]
        # PROCESANDO DATOS SIN ESTRUCTURAS
            program_name = " ".join(user_words[0:user_words.index("Programa:")])
            student_name = " ".join(user_words[user_words.index("Codigo:")+1:user_words.index("Nombre:")])
            student_debt = " ".join(saldo_words[0:saldo_words.index("Vencer:")+1])
            student_vencido = " ".join(saldo_words[saldo_words.index("Vencido:")-1:saldo_words.index("Vencido:")+1])
            data.append({
                        "programa":program_name,
                        "nombre":student_name, 
                        "deuda":int(student_debt.split()[0].replace(",", '')),
                        "vencido":int(student_vencido.split()[0].replace(",", '')),})
        except: 
            doc.close()
            # print(f'{archivo} no se pudo procesar, será eliminado.')
            os.remove(f"../data/uao_ctas/{archivo}")
            del_files+=1
    print(f"Se eliminaron {del_files} archivos.")
    return data

### TEST ZONE

In [8]:
# doc = fitz.open(f"../data/uao_ctas/{2186103}.pdf")
# page1 = doc[0]

#         # una lista de tuplas con las cordenadas de cada palabra y la palabra en si.
# words = page1.get_text("words")

#         # editar el pdf para añadir el contenedor de los saldos.
# x0, y0, x1, y1 = 472,46,502,57
# rect = fitz.Rect(x0-10,y0-20,x1+62,y1)
# page1.draw_rect(rect, color=[0,1,1,0], overlay=False,width=0.5)

#         # editar el pdf para añadir el contenedor del nombre y el programa.

# x02, y02, x12, y12 = 180,95,240,109
# rect2 = fitz.Rect(x02-150,y02-7,x12+350,y12+27)
# page1.draw_rect(rect2, color=[0,1,1,0],width=0.5)

#     # TRABAJAMOS CON LOS DATOS QUE TIENEN RECTANGULOS
# saldo_words_list = [word for word in words if fitz.Rect(word[:4]) in rect]
# user_words_list = [word for word in words if fitz.Rect(word[:4]) in rect2]

# saldo_words = [word[4] for word in saldo_words_list]
# user_words = [word[4] for word in user_words_list]
#     # PROCESANDO DATOS SIN ESTRUCTURAS
# program_name = " ".join(user_words[0:user_words.index("Programa:")])
# student_name = " ".join(user_words[user_words.index("Codigo:")+1:user_words.index("Nombre:")])
# student_debt = " ".join(saldo_words[0:saldo_words.index("Vencer:")+1])

In [9]:
# #2117878

# url1 = "http://wlapp.uao.edu.co/reports/rwservlet?connrepuao_ice&jobtype=jt_uao_ice&report=Estado_Cta_Web_Uao10g.rdf&destype=cache&desformat=pdf&mi_cliente=2040223"
# response1 = requests.get(url1)

# url2 = "http://wlapp.uao.edu.co/reports/rwservlet?connrepuao_ice&jobtype=jt_uao_ice&report=Estado_Cta_Web_Uao10g.rdf&destype=cache&desformat=pdf&mi_cliente=2222159"
# response2 = requests.get(url2)

In [10]:
# archivos = os.listdir("../data/uao_ctas")
# for i in archivos:
#     with open(f"../data/uao_ctas/{i}", "rb") as f:
#             print(f"{i}: {len(f.read())}")
# print(len(response1.content))
# print(len(response2.content))

In [11]:
# def list_splitter(lst: list,n: int) -> list:
#     return [lst[i::n] for i in range(n)]

# codes_splitted = list_splitter(codigos, 1000)

## <b> USANDO EL CODIGO PARA HACER EL ANALISIS

In [12]:
archivos = os.listdir("../data/uao_ctas")
len(archivos)

3484

In [13]:
data = create_data(archivos)

Se eliminaron 358 archivos.


In [14]:
df = pd.DataFrame(data)
df.to_csv("../data/uao_ctas.csv", index=False)